
# Sentiment Classification of Movie Reviews

Sentiment classification is the task of determining whether a given review expresses a positive or negative sentiment. In this project, we will classify movie reviews as either "positive" or "negative" using the IMDb dataset and a pre-trained **Transformers** model, DistilBERT.


## Real-World Use Cases
- **Product Review Analysis:** Understanding customer feedback and identifying the overall sentiment towards products on e-commerce websites.
- **Social Media Monitoring:** Detecting public opinion or sentiment regarding a specific event, product, or political figure.
- **Customer Support Systems:** Automating the process of categorizing customer queries and prioritizing negative sentiment for faster response.

## Reasons for Using Sentiment Classification
- **Improved Decision-Making:** Analyzing customer opinions helps companies make informed decisions regarding products, services, and strategies.
- **Automated Insights:** With a sentiment analysis tool, companies can automatically track the overall sentiment trends without manual effort.
- **Better Customer Experience:** Identifying unhappy customers allows companies to intervene early and offer support.

## Solution Explanation Using Transformers
In this solution, we use **DistilBERT**, a lighter version of BERT (Bidirectional Encoder Representations from Transformers), which is designed for efficient NLP tasks. Transformers are pre-trained language models that learn contextual representations of text. We fine-tune the model on our sentiment classification task using labeled reviews from the IMDb dataset. Here's how the process works:

1. **Data Loading and Tokenization:** We load the IMDb dataset and tokenize the reviews using DistilBERT's tokenizer. Tokenization breaks down the reviews into tokens and converts them into a format that the model can understand.
  
2. **Model Fine-Tuning:** We load the pre-trained DistilBERT model and fine-tune it for sequence classification, where it learns to distinguish between positive and negative reviews.
  
3. **Evaluation:** After training the model, we evaluate its performance on a test set using accuracy and F1-score.
  
4. **Prediction:** Finally, we test the model on a few custom-written reviews to see if it correctly classifies the sentiment.


## Step-by-Step Code Implementation:

#### 1. Install Required Libraries

In [ ]:
!pip install --quiet transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00


#### 2. Import Required Libraries

In [ ]:
# Importing necessary libraries
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

#### 3. Load the IMDb Dataset

In [ ]:
# Load the IMDb dataset
dataset = load_dataset("imdb")
print("Example review from the dataset:")
print(dataset['train'][0]['text'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Example review from the dataset:
I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity sc

In [ ]:
# Display the first 5 rows of the training dataset
print("First 5 rows of the IMDb training dataset:")
for i in range(5):
    print(f"Review {i+1}: {dataset['train'][i]['text']}")
    print(f"Label {i+1}: {'Positive' if dataset['train'][i]['label'] == 1 else 'Negative'}")
    print("-" * 50)

First 5 rows of the IMDb training dataset:
Review 1: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, t

#### 4. Initialize the DistilBERT Tokenizer

In [ ]:
# Use DistilBERT tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#### 5. Tokenize the Dataset

In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
print("Tokenized dataset structure:")
print(tokenized_datasets['train'][0]) # text has been transformed into token IDs and attention masks.

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Tokenized dataset structure:
{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nud

#### 6. Set Dataset Format for PyTorch

In [ ]:
# Set the dataset format to PyTorch
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

print(tokenized_datasets['train'][0])

{'label': tensor(0), 'input_ids': tensor([  101,  1045, 12524,  1045,  2572,  8025,  1011,  3756,  2013,  2026,
         2678,  3573,  2138,  1997,  2035,  1996,  6704,  2008,  5129,  2009,
         2043,  2009,  2001,  2034,  2207,  1999,  3476,  1012,  1045,  2036,
         2657,  2008,  2012,  2034,  2009,  2001,  8243,  2011,  1057,  1012,
         1055,  1012,  8205,  2065,  2009,  2412,  2699,  2000,  4607,  2023,
         2406,  1010,  3568,  2108,  1037,  5470,  1997,  3152,  2641,  1000,
         6801,  1000,  1045,  2428,  2018,  2000,  2156,  2023,  2005,  2870,
         1012,  1026,  7987,  1013,  1028,  1026,  7987,  1013,  1028,  1996,
         5436,  2003,  8857,  2105,  1037,  2402,  4467,  3689,  3076,  2315,
        14229,  2040,  4122,  2000,  4553,  2673,  2016,  2064,  2055,  2166,
         1012,  1999,  3327,  2016,  4122,  2000,  3579,  2014,  3086,  2015,
         2000,  2437,  2070,  4066,  1997,  4516,  2006,  2054,  1996,  2779,
        25430, 14728,  2245,  

#### 7. Split the Dataset for Training and Validation

In [ ]:
# Split the dataset into training and validation sets
train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(10000))  # Use smaller subset for faster training
eval_dataset = tokenized_datasets['test'].shuffle(seed=42).select(range(2000))     # Validation subset

# Print dataset sizes
print(f"Training dataset size: {len(train_dataset)}, Evaluation dataset size: {len(eval_dataset)}")

Training dataset size: 10000, Evaluation dataset size: 2000


#### 8. Define the Evaluation Metric

In [ ]:
# Define a simple evaluation metric function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

#### 9. Load DistilBERT Model

In [ ]:
# Load the DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### 10. Define Training Arguments

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


#### 11. Train the Model

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Print training complete
print("Training complete!")

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.312400,0.213924,0.916500,0.915869,0.922843,0.909000


Training complete!


#### 12. Evaluate the Model

In [ ]:
# Evaluate the model
evaluation_results = trainer.evaluate()

# Print evaluation results
print("Evaluation results:", evaluation_results)

Evaluation results: {'eval_loss': 0.21392443776130676, 'eval_accuracy': 0.9165, 'eval_f1': 0.9158690176322418, 'eval_precision': 0.9228426395939087, 'eval_recall': 0.909, 'eval_runtime': 35.0255, 'eval_samples_per_second': 57.101, 'eval_steps_per_second': 3.569, 'epoch': 1.0}


#### 13. Test the Model with Custom Reviews

In [ ]:
# Test the model on new reviews
test_reviews = [
    "The movie was fantastic, I really enjoyed it!",
    "I hated every moment, it was the worst movie I've seen.",
    "It was an average experience, not too bad but not great either.",
    "I absolutely loved the product, will recommend it to everyone!"
]

In [ ]:
# Tokenize the new reviews
inputs = tokenizer(test_reviews, padding=True, truncation=True, return_tensors="pt")

In [ ]:
# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
# Move model and inputs to the same device
model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}

In [ ]:
# Set the model to evaluation mode
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    predictions = np.argmax(outputs.logits.cpu().numpy(), axis=1)  # Move logits back to CPU for numpy

In [ ]:
# Interpret the results
for i, review in enumerate(test_reviews):
    sentiment = "Positive" if predictions[i] == 1 else "Negative"
    print(f"Review: {review}\nPredicted Sentiment: {sentiment}\n")

Review: The movie was fantastic, I really enjoyed it!
Predicted Sentiment: Positive

Review: I hated every moment, it was the worst movie I've seen.
Predicted Sentiment: Negative

Review: It was an average experience, not too bad but not great either.
Predicted Sentiment: Negative

Review: I absolutely loved the product, will recommend it to everyone!
Predicted Sentiment: Positive

